In [4]:
# kütüphanelerin yüklenmesi
import joblib
import pandas as pd
import os

In [5]:
#model ve encoderin alınması
model_path = '../models/salary_model_target_encoded.pkl'
means_path = '../models/job_title_means.pkl'
global_mean_path = '../models/global_mean_salary.pkl'

loaded_model = joblib.load(model_path)
job_means = joblib.load(means_path)
global_mean = joblib.load(global_mean_path)

In [6]:
def maas_hesapla(yas, tecrube, egitim, kidem, unvan):
    education_map = {
        'unknown': 0,
        'high_school': 1,
        'bachelor': 2,
        'master': 3,
        'phd': 4
    }
    
    # Kıdem haritası
    seniority_map = {
        'junior': 0,
        'senior': 1
    }
    
    # 1) Kategorik verileri sayıya çevir (Mapping)
    egitim_kodu = education_map.get(egitim, 0)
    kidem_kodu = seniority_map.get(kidem, 0)
    
    # 2) Mesleği Target Encoding ile çevir
    # Sözlükten mesleğin ortalamasını bul, yoksa genel ortalamayı (global_mean) al
    unvan_kodu = job_means.get(unvan, global_mean)
    
    # 3) Veriyi DataFrame haline getir
    # Sütun sırası, modeli eğitirken kullandığımız X_train_final ile AYNI olmalı
    yeni_veri = pd.DataFrame([[yas, tecrube, egitim_kodu, kidem_kodu, unvan_kodu]], 
                             columns=['age', 'years_experience', 'education_level_encoded', 'seniority_level_encoded', 'job_title_encoded'])
    
    # 4) Tahmin yap
    tahmin = loaded_model.predict(yeni_veri)[0]
    return tahmin

In [7]:
#deneme
# Senaryo 1: 3 Yıllık Veri Analisti (Data Analyst)
tahmin1 = maas_hesapla(yas=28, tecrube=3, egitim='bachelor', kidem='junior', unvan='data analyst')
print(f"3 Yıllık Data Analyst: ${tahmin1:,.2f}")

# Senaryo 2: 3 Yıllık Satış Temsilcisi (Sales Representative)
tahmin2 = maas_hesapla(yas=28, tecrube=3, egitim='bachelor', kidem='junior', unvan='sales representative')
print(f"3 Yıllık Sales Rep:    ${tahmin2:,.2f}")

3 Yıllık Data Analyst: $66,662.80
3 Yıllık Sales Rep:    $45,195.83
